In [1]:
!pip install -U pyarrow --quiet
!pip install datasets transformers torch  --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00


Categorize call center questions according to topic

In [2]:
import torch
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.model_selection import train_test_split

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load pre-trained model and tokenizer
model_name = "roberta-base"  # Changed from distilbert to roberta
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

def create_sample_dataset():
    sample_data = {
        'text': [
            "I have a question about my latest bill",
            "Why is my bill higher this month?",
            "Can you explain the charges on my invoice?",
            "I want to dispute a charge on my bill",
            "How do I read my billing statement?",
            "When is my next payment due?",
            "Can I change my billing cycle?",
            "My internet connection is not working",
            "I can't connect to the Wi-Fi network",
            "My phone isn't receiving calls",
            "The TV signal keeps cutting out",
            "How do I troubleshoot my router?",
            "My device isn't turning on",
            "I'm experiencing slow internet speeds",
            "How do I change my account password?",
            "Can I update my payment information?",
            "How do I add another user to my account?",
            "I need to reset my account PIN",
            "What are the steps to cancel my subscription?",
            "How do I upgrade my service plan?"
        ] * 15,  # Repeat each question 15 times to increase dataset size
        'label': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2] * 15  # 0: billing, 1: troubleshooting, 2: account_management
    }
    return pd.DataFrame(sample_data)

# Load the dataset
df = create_sample_dataset()

# Split the dataset
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])

# Create datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Prepare dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_tokenized = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
test_tokenized = test_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Set format for PyTorch
train_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_tokenized.set_format("torch", columns=["input_ids", "attention_mask", "label"])

# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted', zero_division=1)
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,  # Increased from 5 to 10
    per_device_train_batch_size=32,  # Increased from 16 to 32
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    learning_rate=5e-5,  # Increased from 2e-5 to 5e-5
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics,
)

eval_results = trainer.evaluate()
print(f"Before Training Evaluation results: {eval_results}")

# Fine-tune the model
trainer.train()

eval_results = trainer.evaluate()
print(f"Post Training Evaluation results: {eval_results}")

# Save the fine-tuned model
trainer.save_model("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

# Load the fine-tuned model for inference
model = AutoModelForSequenceClassification.from_pretrained("./fine_tuned_model").to(device)
model.eval()

# Example of using the fine-tuned model
def classify_query(query):
    inputs = tokenizer(query, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return ["billing", "troubleshooting", "account_management"][predicted_class]

# Test the model
test_queries = [
    "I have a question about my latest bill",
    "My internet connection is not working",
    "How do I change my account password?",
    "Can you explain the charges on my invoice?",
    "The TV signal keeps cutting out",
    "How do I add another user to my account?"
]

for query in test_queries:
    print(f"Query: {query}")
    print(f"Predicted category: {classify_query(query)}\n")

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Before Training Evaluation results: {'eval_loss': 1.099798321723938, 'eval_model_preparation_time': 0.0061, 'eval_accuracy': 0.35, 'eval_f1': 0.18148148148148147, 'eval_precision': 0.7725000000000001, 'eval_recall': 0.35, 'eval_runtime': 3.4423, 'eval_samples_per_second': 17.43, 'eval_steps_per_second': 0.291}


Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Precision,Recall
1,No log,1.099282,0.006100,0.400000,0.271682,0.663636,0.400000
2,1.100100,1.097453,0.006100,0.350000,0.181481,0.772500,0.350000
3,1.097700,1.094252,0.006100,0.400000,0.274176,0.778947,0.400000
4,1.094100,1.088833,0.006100,0.466667,0.353043,0.678125,0.466667
5,1.087000,1.076495,0.006100,0.866667,0.865758,0.887747,0.866667
6,1.087000,1.041957,0.006100,1.000000,1.000000,1.000000,1.000000
7,1.057200,0.896497,0.006100,1.000000,1.000000,1.000000,1.000000
8,0.940700,0.486278,0.006100,1.000000,1.000000,1.000000,1.000000
9,0.539600,0.174933,0.006100,1.000000,1.000000,1.000000,1.000000
10,0.185400,0.046453,0.006100,1.000000,1.000000,1.000000,1.000000


Post Training Evaluation results: {'eval_loss': 1.041957139968872, 'eval_model_preparation_time': 0.0061, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 0.4383, 'eval_samples_per_second': 136.88, 'eval_steps_per_second': 2.281, 'epoch': 10.0}
Query: I have a question about my latest bill
Predicted category: billing

Query: My internet connection is not working
Predicted category: troubleshooting

Query: How do I change my account password?
Predicted category: account_management

Query: Can you explain the charges on my invoice?
Predicted category: billing

Query: The TV signal keeps cutting out
Predicted category: troubleshooting

Query: How do I add another user to my account?
Predicted category: account_management

